# Train and test YOLOv5 model

Install YOLOv5 and dependencies.

In [ ]:
import pprint

print('Setup complete. Using torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

## Config

In [ ]:
# Read config

with open('params.yaml') as conf_file:
    config = yaml.safe_load(conf_file)

pprint.pprint(config)

## Train
### Localization

Install weights and biases to track training progress in real time.

In [ ]:
# Weights & Biases (optional)
%pip install -q wandb
!wandb login  # use 'wandb disabled' or 'wandb enabled' to disable or enable

Train the model.

In [ ]:
%cd /content/yolov5
!wandb enabled
!python train.py --img 128 --batch 64 --epochs 300 --data 'object_data_yaml' --cfg 'object_train_yaml' --weights yolov5m.pt --name train_0076 --cache --hyp 'data/hyp.custom_glasstube.yaml'
%cp -anrv "/content/yolov5/runs/train/train_0076" "/content/gdrive/MyDrive/YOLOv5_working_dir/model/train"

In [ ]:
%cp -anrv "/content/yolov5/runs/train/train_00682" "/content/gdrive/MyDrive/YOLOv5_working_dir/model/train"

### Classification

Download pretrained weights if required.

In [ ]:
%cd /content/yolov5
import utils
from utils.downloads import attempt_download

p5 = ['n', 's', 'm', 'l', 'x']  # P5 models
cls = [f'{x}-cls' for x in p5]  # classification models

for x in cls:
    attempt_download(f'weights/yolov5{x}.pt')

Train.

In [ ]:
%cd /content/yolov5
!wandb enabled
!python classify/train.py --img 64 --batch-size 64 --epochs 200 --data /content/yolov5/split_0038/images --model yolov5m-cls.pt --name train_0067 --cache
%cp -anrv "/content/yolov5/runs/train-cls/train_0067" "/content/gdrive/MyDrive/YOLOv5_working_dir/model/train"

In [ ]:
--pretrained weights/yolov5m-cls.pt

In [ ]:
!python export.py --weights runs/train-cls/train_0058/weights/best.pt --include onnx

Copy training result to google drive.

In [ ]:
%cp -anrv "/content/yolov5/runs/train-cls/train_0067" "/content/gdrive/MyDrive/YOLOv5_working_dir/model/train"

## Test 
### Localization

In [ ]:
%cd /content/yolov5
!python val.py --img 128 --data 'object_data_yaml' --weights /content/gdrive/MyDrive/YOLOv5_working_dir/model/train/train_0076/weights/last.pt --name test_0064 --task 'test' --save-txt --save-conf

Copy test results to google drive.

In [ ]:
%cp -anrv "/content/yolov5/runs/val/test_0064" "/content/gdrive/MyDrive/YOLOv5_working_dir/model/test"

###  Classification

In [ ]:
%cd /content/yolov5
!python classify/val.py --img 64 --data /content/gdrive/MyDrive/split_0038/images --weights /content/gdrive/MyDrive/YOLOv5_working_dir/model/train/train_0067/weights/last.pt --name test_0055

In [ ]:
%cp -anrv "/content/yolov5/runs/val-cls/test_0055" "/content/gdrive/MyDrive/YOLOv5_working_dir/model/test"

## Predict

In [ ]:
%cd /content/yolov5
!python detect.py --source '/content/gdrive/MyDrive/set_0007' --weights '/content/gdrive/MyDrive/YOLOv5_working_dir/model/train/train_0076/weights/last.pt' --img 640 --conf 0.9 --save-txt --name predict_0065

Copy prediction results to google drive.

In [ ]:
%cp -anrv "/content/yolov5/runs/detect/predict_0065" "/content/gdrive/MyDrive/YOLOv5_working_dir/model/predict/"

## Store artifacts

In [ ]:
# Copy things around on google drive
%cp -anrv "/content/gdrive/MyDrive/YOLOv5_working_dir/split/split_0003/images" "/content/gdrive/MyDrive/YOLOv5_working_dir/split/split_0005/images"